In [1]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.duplicate import ElementwiseDuplicateElimination
from pymoo.optimize import minimize
from jobshop.params import JobShopRandomParams, JobShopParams
from jobshop.heurstic.decoder import Decoder
from jobshop.heurstic.brkga import BRKGA

In [2]:
params = JobShopRandomParams(10, 10, t_span=(0, 20), seed=12)  # 10, 10 seed 12 known optimal solution 153

In [3]:
class JobShopProblem(ElementwiseProblem):
    
    def __init__(self, params: JobShopParams):
        self.params = params
        n_var = 0
        for j, machines in self.params.seq.items():
            n_var = n_var + len(machines)
        xl = np.zeros(n_var)
        xu = np.ones(n_var)
        self.decoder = Decoder(
            self.params.machines, self.params.jobs,
            self.params.p_times, self.params.seq
        )
        super().__init__(elementwise=True, n_var=n_var, n_obj=1, xl=xl, xu=xu)
    
    def _evaluate(self, x, out, *args, **kwargs):
        z, C = self.decoder.decode(x)
        out["pheno"] = z
        out["hash"] = hash(str(z))
        out["F"] = C

In [4]:
class DuplicatesEncoder(ElementwiseDuplicateElimination):
    
    def __init__(self, x_tol=1e-3) -> None:
        super().__init__()
        self.x_tol = x_tol

    def is_equal(self, a, b):
        same_pheno = a.get("hash") == b.get("hash")
        diff_x = a.get("X") - b.get("X")
        dist_x = np.sqrt(diff_x.dot(diff_x))
        return same_pheno and dist_x <= self.x_tol * len(diff_x)

In [5]:
brkga = BRKGA(
    pop_size=100,
    perc_elite=0.2,
    perc_mutants=0.15,
    bias=0.8,
    eliminate_duplicates=DuplicatesEncoder(1e-2),
)
problem = JobShopProblem(params)

In [6]:
res = minimize(problem, brkga, ("n_gen", 1000), verbose=True, seed=12)

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |      100 |  5.265847E+02 |  2.280000E+02
     2 |      180 |  4.659390E+02 |  2.180000E+02
     3 |      260 |  4.149106E+02 |  2.180000E+02
     4 |      340 |  4.393598E+02 |  2.180000E+02
     5 |      420 |  4.542290E+02 |  2.180000E+02
     6 |      500 |  4.548790E+02 |  2.180000E+02
     7 |      580 |  4.891428E+02 |  2.180000E+02
     8 |      660 |  4.715109E+02 |  2.170000E+02
     9 |      740 |  4.508317E+02 |  2.170000E+02
    10 |      820 |  4.718209E+02 |  2.170000E+02
    11 |      900 |  4.800855E+02 |  2.170000E+02
    12 |      980 |  4.474217E+02 |  2.090000E+02
    13 |     1060 |  4.478517E+02 |  2.090000E+02
    14 |     1140 |  4.523990E+02 |  2.050000E+02
    15 |     1220 |  4.611236E+02 |  2.050000E+02
    16 |     1300 |  4.319371E+02 |  2.050000E+02
    17 |     1380 |  4.072679E+02 |  2.050000E+02
    18 |     1460 |  4.821728E+02 |  2.050000E+02
    19 |     1540 |  4.636209E+02 |  2.050000E+02


In [ ]:
graph = problem.decoder.build_graph_from_string(res.X)

In [ ]:
graph.plot()